# This notebook will generate the graph representing the urban environment considered in further simulations with a focus on high speed corridors

In [ ]:
from pathlib import Path

"""
We define a default path where the modules developed in USEPE are stored
"""

bluesky_path = Path(Path.home(), 'ws', 'bluesky')

"""
We define a default path where the scenarios are stored
"""

scenarios_path = Path(Path.home(), 'ws', 'scenarios', 'exercise_corridor_focus')


In [ ]:
import os
import configparser
import pickle
import osmnx as ox
import pandas as pd

os.chdir(bluesky_path)
from usepe.city_model.city_graph import cityGraph
from usepe.city_model.corridors_implementation import corridorLoad
from usepe.city_model.dynamic_segments import dynamicSegments
from usepe.city_model.multi_di_graph_3D import MultiDiGrpah3D
from usepe.city_model.utils import read_my_graphml

[City]

mode = rectangle

hannover_lat = 

hannover_lon = 

zone_size = 1000 

hannover_lat_min = 52.373136

hannover_lat_max = 52.427466

hannover_lon_min = 9.771070

hannover_lon_max = 9.859913

import = False

imported_graph_path = data/exe_corridor_focus_ref.graphml


[Layers]

number_of_layers = 9

layer_width = 25


[Corridors]

corridors = 

altitude = 250

delta_z = 25

speed = 40

acceleration_length = 50

file_path_corridors = data/usepe-hannover-corridors.geojson


[Segments]

import = True

imported_segments_file_path = data/Hannover_initial_segments_ref.json


[BuildingData]

lat_min = 52.373136

lat_max = 52.427466

lon_min = 9.771070

lon_max = 9.859913

divisions = 20

directory_hannover = /home/ror/ws/bluesky/usepe/city_model/data/hannover_building_data


[Options]

one_way = False

simplify = True

simplification_distance = 0

simplification_error = 2


[Outputs]

graph_path = data/exe_corridor_focus_ref.graphml


[Strategic_Deconfliction]

ratio = 3

delay = 60


In [ ]:
# -------------- 1. CONFIGURATION FILE -----------------
"""
This section reads the configuration file.
Change the config_path to read the desired file
"""
# CONFIG
config_path = Path(scenarios_path, 'settings_exe_corridor_focus_d2c2.cfg')
config = configparser.ConfigParser()
config.read(config_path)

# City Graph

In [ ]:
# -------------- 2. CITY GRAPH -------------------------
"""
This section creates a city graph or loads the graph defined in the 'City' section of the
configuration file.
"""
# City
os.chdir(scenarios_path)
if config['City'].getboolean( 'import' ):
    filepath = config['City']['imported_graph_path']
    G = read_my_graphml( filepath )
    G = MultiDiGrpah3D( G )
    # fig, ax = ox.plot_graph( G )
else:
    G = cityGraph( config )


# Segments

In [ ]:
# -------------- 3. SEGMENTS ----------------------------
"""
This section creates an airspace segmentation or loads the segmentation defined in the 'Segments'
section of the configuration file.
Comment it to neglect the segmentation
"""
os.chdir(scenarios_path)
if config['Segments'].getboolean( 'import' ):
    segments = pd.read_json(config['Segments']['imported_segments_file_path'], orient="records", lines=True )
else:
    segments = None

# Corridors

In [ ]:
# -------------- 4. CORRIDORS ---------------------------
"""
This section loads the corridors defined in the 'Corridors' section of the configuration file
Comment it to neglect the creation of corridors
"""
G, segments = corridorLoad( G, segments, config )

# Segment update

In [ ]:
# ---------------- 5. Segment update --------------------
"""
Once we have loaded the segments and we created the corridors, we update the graph based on the parameters of the segments
"""
G, segments = dynamicSegments( G, config, segments, deleted_segments=None )

# Saving graph

In [ ]:
os.chdir(scenarios_path)
fig, ax = ox.plot_graph( G )
filepath = config['Outputs']['graph_path']
ox.save_graphml( G, filepath )